In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import sklearn
import nltk
from google.colab import drive

In [2]:
# Import Data
# Mount Google Drive
drive.mount('/content/drive')

# Update file path to the location in your Google Drive
file_path = '/content/drive/MyDrive/spam.csv'

# Read the CSV file
csv = pd.read_csv(file_path, encoding="ISO-8859-1")
df = pd.DataFrame(csv)

Mounted at /content/drive


In [3]:
# check class distribution
classes = df[df.columns[0]]
print(classes.value_counts())

v1
ham     4825
spam     747
Name: count, dtype: int64


Preprocessing the Data

In [4]:
# convert class labels to binary values, 0 = ham  1 = spam

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(classes)

# quick check
print(classes[:10])
print(Y[:10])

0     ham
1     ham
2    spam
3     ham
4     ham
5    spam
6     ham
7     ham
8    spam
9    spam
Name: v1, dtype: object
[0 0 1 0 0 1 0 0 1 1]


In [5]:
# store SMS message data
text_messages = df[df.columns[1]]
print(text_messages[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: v2, dtype: object


In [6]:
# expressions can be found at https://regexlib.com/
# use regular expressions to replace email addresses, urls, phone numbers, etc.

# replace email addresses with 'emailaddr
processed = text_messages.str.replace(r'^\w+@[a-zA-Z_]+?\.[a-zA-Z]{2,3}$', 'emailaddr')

# replace urls with 'webaddress'
processed = processed.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')
# replace money symbols with 'moneysymb'
processed = text_messages.str.replace(r'£|\$', 'moneysymb')

# replace 10 digit phone numbers with 'phonenum'
processed = text_messages.str.replace(r'^[2-9]\d{2}-\d{3}-\d{4}$', 'phonenum')

# replace normal numbers with 'num'
processed = text_messages.str.replace(r'\d+(\.\d+)?', 'num')

In [7]:
# remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

# change all words to lower case
processed = processed.str.lower()
processed.head()

,v2
0,"go until jurong point, crazy.. available only ..."
1,ok lar... joking wif u oni...
2,free entry in 2 a wkly comp to win fa cup fina...
3,u dun say so early hor... u c already then say...
4,"nah i don't think he goes to usf, he lives aro..."


In [8]:

from nltk.corpus import stopwords

# Download the stopwords resource
nltk.download('stopwords')

# Define stop words
stop_words = set(stopwords.words('english'))

# Remove stop words from text messages
processed = processed.apply(lambda x: ' '.join(term for term in x.split() if term not in stop_words))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
# remove word stems using a Porter stemmer
# stemming is the process of reducing a word to its word stem such as removing -ing
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x: ' '.join(
    ps.stem(term) for term in x.split()))

processed.head()

,v2
0,"go jurong point, crazy.. avail bugi n great wo..."
1,ok lar... joke wif u oni...
2,free entri 2 wkli comp win fa cup final tkt 21...
3,u dun say earli hor... u c alreadi say...
4,"nah think goe usf, live around though"


Tokenization

In [10]:
from nltk.tokenize import word_tokenize

# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define stop words
stop_words = set(nltk.corpus.stopwords.words('english'))

# Create bag-of-words
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)

# FreqDist class is used to encode “frequency distributions”, which count the number of times that each outcome of an experiment occurs
all_words_freq = nltk.FreqDist(all_words)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# Print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words_freq.most_common(15)))

# Use the 1500 most common words as features
word_features = list(all_words_freq.keys())[:1500]

Number of words: 72334
Most common words: [('.', 4713), (',', 1872), ('?', 1541), ('!', 1381), ('...', 1131), ('u', 1121), ('&', 916), (';', 764), (':', 717), ('i', 695), ('..', 681), ('call', 642), ("'", 533), (')', 494), ('2', 474)]


In [12]:
# find_features function will determine which of the 1500 word features are contained in the email/message
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features

# example
features = find_features(processed[0])
for key, value in features.items():
    if value == True:
        print(key)

go
jurong
point
,
crazy
..
avail
bugi
n
great
world
la
e
buffet
...
cine
got
amor
wat


In [13]:
# do it for all the messages
messages = list(zip(processed, Y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle(messages)

# call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in messages]

In [14]:
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(featuresets, test_size = 0.25, random_state=seed)
print('Training:',len(training))
print('Testing:',len(testing))

Training: 4179
Testing: 1393


Algorithms:
Scikit-Learn Classifiers with NLTK

In [15]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC

model = SklearnClassifier(SVC(kernel = 'linear'))

# train the model on the training data
model.train(training)

# and test on the testing dataset!
accuracy = nltk.classify.accuracy(model, testing)*100
print("SVC Accuracy: {}".format(accuracy))

SVC Accuracy: 98.63603732950466


In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression()
]

models = zip(names, classifiers)

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model, testing)*100
    print("{} Accuracy: {}".format(name, accuracy))

K Nearest Neighbors Accuracy: 92.10337401292176
Decision Tree Accuracy: 96.62598707824839
Random Forest Accuracy: 98.63603732950466
Logistic Regression Accuracy: 98.49246231155779


In [17]:
# ensemble methods - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression()
]

models = list(zip(names, classifiers))

nltk_ensemble = SklearnClassifier(VotingClassifier(estimators = models, voting = 'hard', n_jobs = -1))
nltk_ensemble.train(training)
accuracy = nltk.classify.accuracy(nltk_model, testing)*100
print("Voting Classifier: Accuracy: {}".format(accuracy))

Voting Classifier: Accuracy: 98.49246231155779


In [18]:
# class label prediction for testing set
txt_features, labels = zip(*testing)

prediction = nltk_ensemble.classify_many(txt_features)

# print a confusion matrix and a classification report
print(classification_report(labels, prediction))

pd.DataFrame(
    confusion_matrix(labels, prediction),
    index = [['actual', 'actual'], ['ham', 'spam']],
    columns = [['predicted', 'predicted'], ['ham', 'spam']])

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      1210
           1       1.00      0.83      0.90       183

    accuracy                           0.98      1393
   macro avg       0.99      0.91      0.95      1393
weighted avg       0.98      0.98      0.98      1393



predicted     
                  ham spam
actual ham       1210    0
       spam        32  151